In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
    
sys.meta_path.append(NotebookFinder())

In [2]:
!pip install pydblite

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable   
from torch import optim
import numpy as np

sys.path.append("../Preprocessor")
import format_module
reviewDB = format_module.FormattedReview.reviewDB

import rnn
import naivebayesian
import cnn
import conclude
import mlp

reviewDB = format_module.ReviewDB("../Preprocessor/pkl/train")
format_module.FormattedReview.setDB(reviewDB)

In [4]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as input:
        return pickle.load(input)

In [5]:
class classifierModule(nn.Module):
    def __init__(self, input_size, batch_size, FRlist, path, refresh = False):
        super(classifierModule, self).__init__()
        self.batch_size = batch_size
        self.input_size = input_size
        
        self.rnn_model = rnn.RNN_model(input_size)
        self.rnn_out_size = rnn.RNN_model.hidden_size
        self.rnn_mlp = mlp.mlp(format_module.FormattedReview.attribute_num, self.rnn_out_size)
        
        self.nb_model = naivebayesian.NaiveBayesianDB()
        self.nb_out_size = 1
        self.nb_mlp = mlp.mlp(format_module.FormattedReview.attribute_num, self.nb_out_size)
        
        self.cnn_model = cnn.ConvNet(input_size)
        self.cnn_out_size = cnn.ConvNet.output_vector_size
        self.cnn_mlp = mlp.mlp(format_module.FormattedReview.attribute_num, self.cnn_out_size)
        
        self.conclude = conclude.conclude()
        
        self.nb_model.add_FRlist(FRlist) #initialize nb database
        
        self.path = path
        
        if os.path.exists(path) and not refresh:
            self.load_state_dict(torch.load(path))
            
        else:
            self.save_state_dict()
        
    def save_state_dict(self):
        torch.save(self.state_dict(), self.path)
        
    def encoder(self, formattedList):
        length = len(formattedList)
        contextList = [formattedList[i].context for i in range(length)]

        lengths = torch.LongTensor([len(contextList[i]) for i in range(length)])
        max_len = torch.max(lengths)
        
        data = np.zeros((length, max_len, self.input_size))

        for i in range(length):
            context = contextList[i]
            if not (context.size == 0):
                data[i, :context.shape[0],:] = context
            else:
                lengths[i] = 1
            
        return self.sort_batch(torch.FloatTensor(data), formattedList, lengths)
        
    def sort_batch(self, context, formatted, seq_len):
        batch_size = context.size(0)
        sorted_seq_len, sorted_idx = seq_len.sort(0, descending = True)
        
        sorted_context = context[sorted_idx]
        sorted_formatted = [formatted[i] for i in sorted_idx]
        
        return Variable(sorted_context), sorted_formatted, sorted_seq_len
    
    def resize_input(self, input):
        list_ = list()
        for i in range(0, len(input), self.batch_size):
            list_.append(input[i:i+self.batch_size])
        return list_
        
    def forward(self, formatted_list, hidden=None, mode = "Default"):
        context, formatted, lengths = self.encoder(formatted_list)
        
        if mode == "rnn":
            rnn_out = self.rnn_model(context, lengths)
            rnn_mlp_out = self.rnn_mlp(self.rnn_mlp.getdata(formatted, rnn_out))
            output_0to1 = torch.nn.functional.sigmoid(rnn_mlp_out)
            return torch.cat([1- output_0to1, output_0to1],1)
            
        elif mode == "cnn":
            cnn_out = self.cnn_model(context)
            cnn_mlp_out = self.cnn_mlp(self.cnn_mlp.getdata(formatted, cnn_out))
            output_0to1 = torch.nn.functional.sigmoid(cnn_mlp_out)
            return torch.cat([1- output_0to1, output_0to1],1)
            
        elif mode == "nb":
            nb_out = self.nb_model.naive_bayes_FRlist(formatted)
            nb_mlp_out = self.nb_mlp(self.nb_mlp.getdata(formatted, nb_out))
            output_0to1 = torch.nn.functional.sigmoid(nb_mlp_out)
            return torch.cat([1- output_0to1, output_0to1],1)
            
        else:
            rnn_out = self.rnn_model(context, lengths)
            cnn_out = self.cnn_model(context)
            nb_out = self.nb_model.naive_bayes_FRlist(formatted)
            
            rnn_mlp_out = self.rnn_mlp(self.rnn_mlp.getdata(formatted, rnn_out))
            cnn_mlp_out = self.cnn_mlp(self.cnn_mlp.getdata(formatted, cnn_out))
            nb_mlp_out = self.nb_mlp(self.nb_mlp.getdata(formatted, nb_out))
            
            return self.conclude(self.conclude.bind(rnn_mlp_out, cnn_mlp_out, nb_mlp_out))
        
        '''
        print("rnn_out : ", rnn_out.size())
        print("cnn_out : ", cnn_out.size())
        print("nb_out : ", nb_out.size(), "\n")
        '''
        
    def print_contribution(self):
        (weight, bias) = self.conclude.get_contribution()
        print("----------------- Current model contribution ----------------")
        print("-- rnn : ", weight.data[0][0])
        print("-- cnn : ", weight.data[0][1])
        print("-- nb : ", weight.data[0][2])
        print("-- bias : ", bias.data[0])
        print("-------------------------------------------------------------\n")

# Training

### hyperparameters :

1. learning_rate
2. input_size
3. rnn_output_size, cnn_output_size
4. batch_size
5. optimizer
6. loss function
7. n_epochs

In [ ]:
learning_rate = 0.005
input_size = 100  # word2vec k size
batch_size = 100
n_epochs = 40

In [7]:
# target이 0일 때, p가 1-s보다 작으면 +1
# target이 1일 때, p가 1-s보다 크면 +1
# -> (1-s-p)*(t-1/2) <= 0 일 때 +1
def get_accuracy(outputs, targets, sensitivity):
    result = 0
    t = targets.data.type(torch.FloatTensor)-0.5
    x = (1-sensitivity-outputs.data[:, 1])*t
    for y in x:
        if y < 0:
            result+=1
    return result
    
def get_targets(input, model):
    _, batch, _ = model.encoder(input)
    targets = list()
    for formatted in batch:
        if formatted.label:
            targets.append(1)
        else:
            targets.append(0)
            
    return Variable(torch.LongTensor(targets), requires_grad = False)

def get_prediction(outputs, sensitivity):
    return np.ceil(outputs.data[:, 1]+sensitivity-1+0.000000001)

In [8]:
def train_net(train_list, validation_list, sensitivity = 0.5, run_mode = "default"):
    batch_list = model.resize_input(train_list)
    
    tacc_list = list()
    vacc_list = list()
    
    for input in train_list:
        reviewDB.add_spam_result(input.bookingReview.id, input.label)
    
    for epoch in range(n_epochs):
        tacc_list.append(0)
        vacc_list.append(0)
        
        for bl in batch_list:
            outputs = model(bl, mode = run_mode)
            targets = get_targets(bl, model)

            optimizer.zero_grad()
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            tacc_list[-1] += get_accuracy(outputs, targets, sensitivity)
            
        tacc_list[-1] = tacc_list[-1] / len(train_list)
        
        v_outputs = model(validation_list, mode = run_mode)
        v_targets = get_targets(validation_list, model)
        vacc_list[-1] = get_accuracy(v_outputs, v_targets, sensitivity) / len(validation_list)
        v_loss = criterion(v_outputs, v_targets)
    
        print("epoch {}: train acc {:.6f} | validation acc {:.6f}" 
              .format(epoch, tacc_list[-1], vacc_list[-1]))
        print("------- loss.data   {:.6f} | v_loss.data    {:.6f}"
              .format(loss.data[0], v_loss.data[0]))
        
        #if epoch > 5 and np.mean(np.array(tacc_list[-6:-1])) < np.mean(np.array(vacc_list[-6:-1])):
        #    print("Seems like m1 starts to overfit, aborting training")
        #    break
            
        model.save_state_dict()
            
    print("Finished Training")

spamFilterModule, which incorporates both preprocessing and classifying.


## Code from
https://github.com/szagoruyko/functional-zoo/blob/master/visualize.py